## OSC(online schema change)가 뭐죠?

서비스 중인 프로덕션 환경의 테이블 스키마를 수정할 때는, 스키마를 수정하는 도중에도 데이터를 READ / WRITE 할 수 있어야 한다. 이렇게  무중단으로 스키마를 수정하는 방법을 Online-Schema-Change, OSC라 한다.


## OSC를 사용하게 된 배경...

alter table로 프로덕션 DB를 수정하다가, table lock으로 인해 일부 테이블 사용이 불가능해진 사고가 발생했다. 문제가 생긴 테이블들은 drop, create 모두 불가능한 교착 상태가 지속되었고 해결에 상당한 시간이 소요되었다. 스키마 변경에 따른 DB 문제 재발 방지를 위해 OSC를 도입하였다.

## 개념 설명

기존 `alter table` 방식

1. 해당 테이블을 락으로 새로운 `READ` or `WRITE` 차단 ( READ / WRITE lock 여부는 DB 엔진, DB 버전, 쿼리문마다 얘기가 다르므로 주의) - [mysql5.6]([https://dev.mysql.com/doc/refman/5.6/en/innodb-online-ddl-operations.html](https://dev.mysql.com/doc/refman/5.6/en/innodb-online-ddl-operations.html))
2. 새로운 테이블의 빈 임시 테이블을 만들고, (스키마 변경이 반영된)
3. 기존 테이블에서 새 테이블에 데이터를 복사하고,
4. 새 테이블을 기존 테이블과 같은 이름으로 바꾸고 기존 테이블을 삭제한다.
5. 새로운 테이블에 차단했던 `Lock`을 푼다. 

문제점 : 락 문제때문에 프로덕션에선 사용하기 곤란하다. 락이 걸린 동안 발생하는 READ / WRITE가 불가능해지고, 데드락으로 인한 DB 교착이 생길 수 있다.

그래서 어떻게? 락 걸지마! - `OSC` 의 해결 방법

- `alter` 프로세스를 스크립트화하여, 사용자(DBA)가 컨트롤할 수 있게 하였다. 새 테이블을 생성해서 복붙 후 기존 테이블 삭제하는 로직은 동일하다.
- `Lock`을 거는 대신 `Trigger`, `binary log stream`  등을 이용하여, 새 테이블로 데이터 이동 중에도 계속해서 새로운 READ/WRITE 가능토록 했다. 서비스가 지속가능하다.
- 무중단으로 스키마가 변경되었다. Profit!!!

## OSC 사용 방법

하여 OSC의 대표적인 툴인 percona에서 만든, pt-online-schema-change를 소개한다.

각자 환경에 맞는 방법으로 percona-toolkit을 설치한다. (글에서는 mac homebrew로 설치)

```bash
brew install percona-toolkit
```

pt-online-schema-change 스크립트를 환경에 맞게 옵션을 설정하여 셸에서 실행하면 스키마 변경이 진행된다. 다음과 같은 상황에서 사용한다.

- 테이블 컬럼 추가, 변경
- 각종 key 설정(constraint, pk 등)

테스트용 셸 스크립트

```bash
pt-online-schema-change \
--alter "add column testcol varchar(255) default null" D=PetRegistration,t=test_A \
--chunk-size=200 \
--host=petregistrantion-test.crt3wdpmhtxz.ap-northeast-2.rds.amazonaws.com \
--port=3306 \
--user=peopet \
--password="impactor123#" \
--progress=time,30 \
--charset=UTF8 \
--execute
```

라이브용 셸 스크립트

- 참고자료
- [https://jojoldu.tistory.com/358](https://jojoldu.tistory.com/358)
- [https://gywn.net/2017/08/small-talk-pt-osc/](https://gywn.net/2017/08/small-talk-pt-osc/)
- [https://blog.myungseokang.dev/posts/online-schema-change/](https://blog.myungseokang.dev/posts/online-schema-change/)